In [1]:
print("Hello World")

Hello World


In [4]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [5]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")


In [ ]:
import asyncio
agent_1 = AssistantAgent(name="georgraphy_expert", model_client=model_client)
agent_2 = AssistantAgent(name="math_expert", model_client=model_client)
agent_3 = AssistantAgent(name="impossible_agent", model_client=model_client)

tasks = [
    agent_1.run(task="What is the capital of France?"),
    agent_2.run(task="What is 2 + 2?"),
    agent_3.run(task="Solve world hunger in 5 words")
]

results = await asyncio.gather(*tasks, return_exceptions=True)
print(results)

TypeError: gather() got an unexpected keyword argument 'semaphore'

In [12]:
async def test_timeout():
    try:
        result = await asyncio.wait_for(
            agent_1.run(task="Write a 10000-word essay"),
            timeout = 0.1
        )
    except asyncio.TimeoutError:
        print("Task timed out")
    except Exception as e:
        print(f"An error occurred: {e}")
    else:
        print(f"Task completed with result: {result}")

await test_timeout()

Task timed out


In [13]:
try:
    bad_client = OpenAIChatCompletionClient(model="kahipan", api_key="bad_key")
    bad_agent = AssistantAgent(name="bad_agent", model_client=bad_client)
    result = await bad_agent.run(task="What is the capital of France?")
except Exception as e:
    print(f"An error occurred: {e}")
else:
    print(f"Task completed with result: {result}")

An error occurred: model_info is required when model name is not a valid OpenAI model


In [22]:
import asyncio
import time

MAX_CONCURRENT_TASKS = 20
semaphore = asyncio.Semaphore(MAX_CONCURRENT_TASKS)

async def run_agent_with_limit(agent, task):
    async with semaphore:
        return await agent.run(task=task)
    
tasks = []

for i in range(10):
    agent = AssistantAgent(name=f"agent_{i}", model_client=model_client)
    task = run_agent_with_limit(agent, f"What is {i} + 1?")
    tasks.append(task)

start_time = time.time()
results = await asyncio.gather(*tasks)
end_time = time.time()
    
print(f"Time taken: {end_time - start_time} seconds for total {len(tasks)} tasks for {MAX_CONCURRENT_TASKS} concurrent tasks")



/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py:139: RuntimeWarning: coroutine 'test_timeout' was never awaited
  return _CRLock(*args, **kwargs)


Time taken: 0.6929941177368164 seconds for total 10 tasks for 20 concurrent tasks


### Time taken: 6.659104108810425 seconds for total 10 tasks for 1 concurrent tasks
### Time taken: 1.3516910076141357 seconds for total 10 tasks for 5 concurrent tasks
### Time taken: 0.9615240097045898 seconds for total 10 tasks for 10 concurrent tasks
### Time taken: 0.6929941177368164 seconds for total 10 tasks for 20 concurrent tasks


In [23]:
agents = []
for i in range(10):
    agent = AssistantAgent(name=f"agent_{i}", model_client=model_client)
    agents.append(agent)

start_time = time.time()

results_unlimited = await asyncio.gather(*[
    agent.run(task=f"What is {i} + 1?")
    for i, agent in enumerate(agents)   
])

unlimited_time = time.time() - start_time

start_time = time.time()

results_limited = await asyncio.gather(*[
    run_agent_with_limit(agent, f"What is {i} + 1?")
    for i, agent in enumerate(agents)
])

limited_time = time.time() - start_time

print(f"Time taken for unlimited concurrent tasks: {unlimited_time} seconds")
print(f"Time taken for limited concurrent tasks: {limited_time} seconds")

Time taken for unlimited concurrent tasks: 0.6943092346191406 seconds
Time taken for limited concurrent tasks: 1.3454647064208984 seconds


In [25]:
import asyncio
import time

MAX_CONCURRENT_TASKS = 3
semaphore = asyncio.Semaphore(MAX_CONCURRENT_TASKS)

  # Progress tracking
progress = {"completed": 0, "total": 50, "errors": 0, "successes": 0}

async def run_agent_with_limit(agent, task, task_id):
    async with semaphore:
        try:
            print(f"Starting task {task_id + 1}/{progress['total']}")
            result = await agent.run(task=task)

            # Update progress
            progress["completed"] += 1
            print(f"✅ Task {task_id + 1} completed ({progress['completed']}/{progress['total']})")

            return {"success": True, "task_id": task_id, "result": result}
        except Exception as e:
            progress["completed"] += 1
            print(f"❌ Task {task_id + 1} failed ({progress['completed']}/{progress['total']}): {e}")
            return {"success": False, "task_id": task_id, "error": str(e)}

  # Create tasks
tasks = []
for i in range(50):
    agent = AssistantAgent(name=f"agent_{i}", model_client=model_client)
    task = run_agent_with_limit(agent, f"What is {i} + 1?", i)
    tasks.append(task)

# Run with timing
start_time = time.time()
results = await asyncio.gather(*tasks)
end_time = time.time()

# Analyze results
successful = [r for r in results if r["success"]]
failed = [r for r in results if not r["success"]]

print(f"\n🎉 Summary:")
print(f"Total time: {end_time - start_time:.2f} seconds")
print(f"Successful: {len(successful)}")
print(f"Failed: {len(failed)}")

if failed:
    print(f"\n❌ Failed tasks:")
    for f in failed:
        print(f"  Task {f['task_id']}: {f['error']}")

Starting task 1/50
Starting task 2/50
Starting task 3/50
✅ Task 2 completed (1/50)
Starting task 4/50
✅ Task 1 completed (2/50)
✅ Task 3 completed (3/50)
Starting task 5/50
Starting task 6/50
✅ Task 5 completed (4/50)
Starting task 7/50
✅ Task 6 completed (5/50)
Starting task 8/50
✅ Task 4 completed (6/50)
Starting task 9/50
✅ Task 8 completed (7/50)
Starting task 10/50
✅ Task 7 completed (8/50)
Starting task 11/50
✅ Task 9 completed (9/50)
Starting task 12/50
✅ Task 10 completed (10/50)
Starting task 13/50
✅ Task 11 completed (11/50)
Starting task 14/50
✅ Task 12 completed (12/50)
Starting task 15/50
✅ Task 13 completed (13/50)
Starting task 16/50
✅ Task 14 completed (14/50)
Starting task 17/50
✅ Task 15 completed (15/50)
Starting task 18/50
✅ Task 16 completed (16/50)
Starting task 19/50
✅ Task 18 completed (17/50)
Starting task 20/50
✅ Task 19 completed (18/50)
Starting task 21/50
✅ Task 17 completed (19/50)
Starting task 22/50
✅ Task 20 completed (20/50)
Starting task 23/50
✅ Task 